# Required Files

Detex requires two files: the station key and the template key (referred to as "keys"). Both are usually saved as csv files for ease of editing/viewing.

## Station Key

The station key is used to tell detex which stations and channels to use. The location of the station and a desired time frame are also included. 

Here is an example from the introductory tutorial:


| NETWORK | STATION | STARTTIME | ENDTIME | LAT | LON | ELEVATION | CHANNELS |
|:-------:|:-------:|:---------:| :-----: | :-: | :-: | :-------: | :------: |
| TA | M18A	| 2009-04-01T00:00:00 | 2009-04-04T00:00:00	| 41.4272 | -110.0674 | 2103 | BHE-BHN-BHZ |
| TA | M17A	| 2009-04-01T00:00:00 | 2009-04-04T00:00:00	| 41.4729 | - 110.6664 | 2101 | BHE-BHN-BHZ |


The STARTTIME and ENDTIME fields indicate the time range of the continuous data and can be in any format readable by the [obspy.UTCDateTime class](http://docs.obspy.org/packages/autogen/obspy.core.utcdatetime.UTCDateTime.html) including a time stamp (ie epoch time). If you do use a time stamp be careful when editing outside of python because some programs, like excel, tend to silently round large numbers. See the [obspy.UTCDateTime docs](https://docs.obspy.org/packages/autogen/obspy.core.utcdatetime.UTCDateTime.html) for more info on readable formats. 

The CHANNELS field lists the channels that will be used for each station. If multiple channels are used they are separated by a dash (-).

The LAT, LON, and ELEVATION fields give the stations location in global coordinates (elevation is from sea-level, in meters).

The order of the headers is not important. Extra fields can be added without affecting detex's ability to read the file. If you need to keep track of location for example, simply add a location field.

## Template Key

The template key is usually saved as TemplateKey.csv. It contains information on each of the events that will be used by detex. 

Here is an example from the introductory tutorial:

| CONTRIBUTOR | NAME | TIME | LAT | LON | DEPTH | MTYPE | MAG |
| :---------: | :--: | :--: | :-: | :-: |:----: | :---: |:--: |
| ANF | 2007-12-19T17-56-18 | 2007-12-19T17-56-18 | 41.7205	| -110.6486	| 4.07 | ML | 2.36 |
| ANF | 2007-12-21T18-30-09	| 2007-12-21T18-30-09 | 41.7669	| -110.6122	| 8.97 | ML | 2.17 |
| ANF | 2007-12-21T18-30-09	| 2007-12-21T18-30-09 | 41.7669	| -110.6122	| 8.97 | ML	| 2.17 |

The NAME field can be any string that can also be used as a file name by your OS. Windows does not allow ":" in a file path so the ":" between the hour and minute, and between the minute and seconds, have been replaced with a "-".

The TIME field, just like the STARTTIME and ENDTIME fields in the station key, can be in any obspy UTCDateTime readable format. 

The MAG field is used in estimating magnitudes of newly detected events. 

The LAT, LON, and DEPTH fields are used in some visualization methods. 

The CONTRIBUTOR and MTYPE fields are not required by detex but can be useful for record keeping. Additionally, just as with the station key, any extra fields can be added in any order. 

# Generating Keys

As long as the comma separated format shown above is followed you can use any method you like to create the keys. For small data sets it may be suitable to create the keys by hand in a text editor or in a program like open office. For larger data sets, however, it is better to either use some of the built in functions to generate the keys or create your own script to do so. 

The following shows a few of the built in methods for generating the keys but it is an good learning exercise in python, especially for those new in the language, to generate these files yourself. If you do write a script or function that uses some data source detex currently cannot read consider contributing it to detex as others will probably find it useful.

The following examples follows the [obspy FDSN tutorial](https://docs.obspy.org/packages/obspy.fdsn.html) closely. 



## Generating Station Keys

The format for the station key is very similar to that produced by the [IRIS station query](https://ds.iris.edu/SeismiQuery/station.htm). If you elect to have the results emailed to you it becomes a trivial to make a station key from the data in the email. Currently there is only one method to make the station key which uses an instance of the [obspy Inventory class](https://docs.obspy.org/packages/autogen/obspy.station.inventory.Inventory.html) as an input argument. 

### Station key from obspy inventory object


In [ ]:
import detex
import obspy
from obspy.fdsn import Client
import obspy

client = Client("IRIS") # use IRIS client

starttime = obspy.UTCDateTime('2009-01-01')
endtime = obspy.UTCDateTime('2010-01-01')

lat = 41.4272
lon = -110.0674

inv = client.get_stations(network="TA", starttime=starttime, endtime=endtime, 
                          channel='BH*', latitude=lat, longitude=lon, maxradius=1,
                         level='channel')


In [ ]:
stakey = detex.util.inventory2StationKey(inv, starttime, endtime)
stakey

## Generating Template Keys

There are two methods for generating template keys. The first uses an obspy catalog object as input and the second uses the output from the University of Utah Seismograph Stations (UUSS) code EQsearch. 

### Template key from obspy catalog object

In [ ]:
usgs = Client("USGS")
cat = usgs.get_events(starttime=starttime, endtime=endtime, minmagnitude=2.5,
                        latitude=lat, longitude=lon, maxradius=2)

In [ ]:
cat

to use this catalog as a template key we simply need to call the catalog2TemplateKey function of detex.util

In [ ]:
temkey = detex.util.catalog2Templatekey(cat) # get template key as DataFrame
temkey.to_csv('TemplateKey.csv', index=False) # save as csv

In [ ]:
temkey

The other function used to make a template key is only useful at the University of Utah where a program called EQsearch is used to query the UUSS catalog. EQsearch produces a file, by default, called eqsrchsum. The function EQSearch2TemplateKey is a parser that takes the information from this file and converts it to a template key.

# Reading Keys

All detex functions and classes that use a key file call the detex.util.readKey function to read in the key file (either template key, station key, or phase picks) or to validate a key that is already in memory (in the form of a DataFrame). This function makes sure all the required fields exist and have legal values. If you want to verify that a key file you have created is valid simply try and read it in with the readKey function. Alternatively, we can pass a DataFrame to the function to see if it is a valid key. 

In [ ]:
temkey2 = detex.util.readKey(temkey, key_type='template')
stakey2 = detex.util.readKey(stakey, key_type='station')

Since no errors were raised the station key and the template key we created are valid.

# Next Section
The [next section](../GetData/get_data.md) covers how to get data from various sources. 